## Урок 7. Открытые данные

1. Создать приложение, которое будет из готового файла с данными «Сбербанка» выводить результат по параметрам:

Тип данных

Интервал дат

Область

In [ ]:
import matplotlib.pyplot as plt
import csv
import datetime

data_type = 'Средняя зарплата'
date1 = datetime.date(2019, 8, 1)
date2 = datetime.date(2019, 8, 30)
area = 'Москва'

with open('opendata.csv','r') as f:
    reader=csv.DictReader(f)
    field_names = reader.fieldnames
    print(field_names)
    money = []
    date = []
    for row in reader:
        if (row['name'] == data_type and
        row['region'] == area):
            date.append(row['date'])
            money.append(row['value'])

plt.plot(date,money)
plt.show()


2. Визуализировать выводимые данные с помощью графика